In [1]:
import numpy as np
import pandas as pd

from multiprocessing import Pool

from model import PModel, PIP

### Define functions to calculate diversity metrics and the slope from a quadratic polynomial

In [2]:
def shannon_div(X_t):
	eq_prop = X_t[:,-1] / np.sum(X_t[:,-1])
	eq_prop = eq_prop[eq_prop > 0]
	div = -np.sum(eq_prop*np.log(eq_prop))
	return div

def nucleotide_div(model, X_t):
	div = 0
	sum = np.sum(X_t[:, -1])
	
	for j in range(0, model.S_genotypes):
		for k in range(j + 1, model.S_genotypes):
			pi = np.sum(np.absolute(model.G[j] - model.G[k]))
			div += (X_t[j, -1]*X_t[k, -1]*pi) / sum**2
			
	return div

def get_slope(model):
	model.beta = 1
	model.normalize()
	
	res, fec = model.pareto()
	slope = np.polyfit(res, 1-fec, 2)[0]

	return slope

### Function to run a simulation and return a dictionary of summary values for the simulation run

In [3]:
def get_sim_results(epistasis=0, n_loci=9, beta=0.005, n_gens = 15, t = (0, 1000), p1 = 0.3, p2 = 0.3, sigma1 = 0.3, sigma2 = 0.3):
    cost = np.random.exponential(0.1, n_loci)
    res = np.random.exponential(0.1, n_loci)

    model = PModel(n_loci, res, cost, beta=beta)

    if epistasis == 1:
        model.add_epistasis(2, p1, sigma1)
    elif epistasis == 2:
        model.add_epistasis(2, p1, sigma1)
        model.add_epistasis(3, p2, sigma2)

    model.normalize()

    data = {}
    
    X_t, _ = model.run_sim(t, n_gens)

    threshold = 1
    
    #Check for polymorphism and update counters
    if np.sum(X_t[:, -1] > threshold) > 1:
        data['Polymorphism'] = True
    else:
        data['Polymorphism'] = False

    res_pareto, fec_pareto = model.pareto()
    res_poly, fec_poly, _ = model.poly_approx(order=3, points=1000)
    res_linear, fec_linear = model.linear_approx(points=1000)

    data['PIPMin: Pareto'] = np.min(np.sum(PIP(res_pareto, fec_pareto), axis=1))
    data['PIPMin: Polynomial'] = np.min(np.sum(PIP(res_poly, fec_poly), axis=1))
    data['PIPMin: Linear'] = np.min(np.sum(PIP(res_linear, fec_linear), axis=1))
    
    data['Shannon Diversity'] = shannon_div(X_t)
    data['Nucleotide Diversity'] = nucleotide_div(model, X_t)
    data['Slope'] = get_slope(model)

    return data


### Run a batch of simulations, and save the outputs as a .csv file

In [5]:
from numpy.random import seed

n_sims = 1000
epistasis = 0
name = 'epi_0'

def init_pool_processes():
    seed()

pool = Pool(processes=8, initializer=init_pool_processes)
results = pool.map(get_sim_results, np.ones(n_sims)*epistasis)

data = pd.DataFrame.from_dict(results, orient='columns')
data.to_csv(name + '.csv', index=False, mode='w')  